<img src="atom.png" alt="Atom" style="width:60px" align="left" vertical-align="middle">

## 1. Using Weak References to list instances
*in Python 3*

----
Python does not keep track of all instances of a class (or of a built-in type), so if you need this functionality, you have to implement it yourself. One way to do it is to store a weak reference to each instance in class attribute. Here’s an example:

In [19]:
import weakref

class MyClass:
    _instances = set()
    
    def __init__(self, name):
        self.name = name
        self._instances.add(weakref.ref(self))

    @classmethod
    def GetInstances(cls):
        dead = set()
        for ref in cls._instances:
            obj = ref()
            if obj is not None: yield obj
            else: dead.add(ref)
        cls._instances -= dead

a = MyClass("a")
b = MyClass("b")
c = MyClass("c")
del b

# Prints 'a' and 'c' as 'b' has been deleted
for obj in MyClass.GetInstances(): print(obj.name)

c
a


<img src="atom.png" alt="Atom" style="width:60px" align="left" vertical-align="middle">

## 2. Using Weak References to list instances
*in Python 3*

----
Alternatively, you can append instances to `self` (althought this method does not take into account objects that have been deleted):

In [17]:
class MyClass:
    instances = []
    def __init__(self, name):
        self.__class__.instances.append(self)
        self.name = name

    @classmethod
    def GetInstances(cls):
        return [instance for instance in cls.instances]
    
a = MyClass("a")
b = MyClass("b")
c = MyClass("c")
del b

# Prints 'a', 'b' and 'c' eventhough 'b' has been deleted
for obj in MyClass.GetInstances(): print(obj.name)

a
b
c


<img src="atom.png" alt="Atom" style="width:60px" align="left" vertical-align="middle">

## 3. Using Python garbage collector
*in Python 3*

----
A simpler, brute-force approach is to ask the `gc` (garbage collector) module for a list of all known objects, and filter out the objects that belong to the relevant class:

In [23]:
import gc

class MyClass:
    def __init__(self, name):
        self.name = name

a = MyClass("a")
b = MyClass("b")
c = MyClass("c")
del b
        
for obj in gc.get_objects():
    if isinstance(obj, MyClass): 
        print(obj.name)

a
c


This has the disadvantage of being very slow when you have a lot of objects, but works with types over which you have no control.